# Imports

In [ ]:
import cairo
import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pickle
import sys

from PIL import Image
from rembg.bg import remove as remove_background
from skimage.draw import line

# Parameters

In [ ]:
# Parameters 

# Program parameters
CLEAN_WORKSPACE = False # Clean all output files
VISUALIZE_DURING_RUN = True # Visualize the results during the run

# I/O path parameters
INPUT_IMAGE_PATH = "img/taco.png"
OUTPUT_IMAGE_PATH = "img_out/" + os.path.splitext(INPUT_IMAGE_PATH[4:])[0]

# Input image preprocessing parameters
CONTRAST_FACTOR = 1.5
INVERSE_INPUT_IMAGE = False
MASK_BACKGROUND_FOCUS = 1
MASK_OBJECT_FOCUS = 1
MASK_EDGES_FOCUS = 10

# Iterations parameters
PREFERED_DARKNESS = 1.0
SAVE_AROUND = 200 # save this amount of iterations above and below the calculated iteration
START_SAVE = 1000
SAVE_INTERVAL = 500 # Save the results every SAVE_INTERVAL iterations
MAX_ITERATIONS = 6000 # Maximum number of iterations 
ITERATION_MODE = "interval" # Can be set to: "auto" / "interval"


# Visual parameters
NAIL_ARRANGEMENT = "rectangle" # Can be set to: "ellipse" / "rectangle"

In [ ]:
#
# Clean workspace
#
def clean_workspace():
    for filename in os.listdir("./img_out_txt"):
        if filename.endswith(".txt") or filename.endswith(".png") or filename.endswith(".svg"):
            os.remove(os.path.join("./img_out_txt", filename))
    for filename in os.listdir("./img_out_png"):
        if filename.endswith(".png"):
            os.remove(os.path.join("./img_out_png", filename))
    for filename in os.listdir("./img_out_svg"):
        if filename.endswith(".svg"):
            os.remove(os.path.join("./img_out_svg", filename))

#
# Display 
#
def display_image(image, rotate=False):
    if rotate:
        image = cv2.rotate(image, cv2.ROTATE_90_COUNTERCLOCKWISE)
    display(Image.fromarray(image))

def display_mask(image, rotate=False):
    canvas = (image / (image.max() / 255)).astype(np.uint8)
    if rotate:
        canvas = cv2.rotate(canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)
    display(Image.fromarray(canvas))

def display_nails(nails, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for nail in nails:
        canvas[nail[0], nail[1]] = 0
    display_image(canvas, rotate)
    return canvas

def display_threads(threads, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for thread in threads:
        canvas = cv2.line(canvas, (thread[0][1], thread[0][0]), (thread[1][1], thread[1][0]), 0, 1)
    display_image(canvas, rotate)
    return canvas

#
# Save
#
def get_save_id(type, content=None):
    na_str = "e_"
    if NAIL_ARRANGEMENT == "rectangle": na_str = "r_"

    c_str = "c" + str(int(CONTRAST_FACTOR * 10)) + "_"

    inv_str = "inv0_"
    if INVERSE_INPUT_IMAGE: inv_str = "inv1_"

    bg_str = "bg" + str(MASK_BACKGROUND_FOCUS) + "_"
    obj_str = "obj" + str(MASK_OBJECT_FOCUS) + "_"
    ed_str = "ed" + str(MASK_EDGES_FOCUS)

    if type == "png":
        png_base_str = OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_"
        i_str = "i" + str(len(content)) + "_"
        return png_base_str + na_str + i_str + c_str + inv_str + bg_str + obj_str + ed_str + ".png"
    elif type == "svg":
        svg_base_str = OUTPUT_IMAGE_PATH[:7] + "_svg" + OUTPUT_IMAGE_PATH[7:] + "_"
        i_str = "i" + str(len(content)) + "_"
        return svg_base_str + na_str + i_str + c_str + inv_str + bg_str + obj_str + ed_str + ".svg"
    elif type == "txt":
        txt_base_str = OUTPUT_IMAGE_PATH[:7] + "_txt" + OUTPUT_IMAGE_PATH[7:] + "_"
        i_str = "i" + str(len(content.split())-1) + "_"
        return txt_base_str + na_str + i_str + c_str + inv_str + bg_str + obj_str + ed_str + ".txt"
    elif type == "mask":
        mask_base_str = OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_mask_"
        return mask_base_str + c_str + inv_str + bg_str + obj_str + ed_str + ".png"
    elif type == "original":
        mask_base_str = OUTPUT_IMAGE_PATH[:7] + "_png" + OUTPUT_IMAGE_PATH[7:] + "_original_"
        return mask_base_str + c_str + inv_str[:4] + ".png"
    
    return "error.txt"

def save_threads(threads, image, rotate=False):
    canvas = np.ones(image.shape, np.uint8) * 255
    for thread in threads:
        canvas = cv2.line(canvas, (thread[0][1], thread[0][0]), (thread[1][1], thread[1][0]), 0, 1)
    if rotate:
        canvas = cv2.rotate(canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)
    print("Trying to save image as: " + get_save_id("png", threads))
    cv2.imwrite(get_save_id("png", threads), canvas)

def save_threads_svg(threads, size, rotate):
    print("Trying to save image as: " + get_save_id("svg", threads))
    with cairo.SVGSurface(get_save_id("svg", threads), size[1], size[0]) as surface:
        context = cairo.Context(surface)
        context.set_source_rgb(255, 255, 255)
        context.rectangle(0, 0, size[1]-1, size[0]-1)
        context.fill()
        context.set_source_rgb(0, 0, 0)
        context.set_line_width(1)
        for thread in threads:
            context.move_to(thread[0][0], thread[0][1])
            for i in range(1, len(thread)):
                context.line_to(thread[i][0], thread[i][1])
            context.stroke()
        if rotate:
            context.rotate(np.pi/2)
            context.translate(0, -size[0])


def save_threads_path(threads):
    print("Trying to save path string as: " + get_save_id("txt", threads))
    with open(get_save_id("txt", threads), "w") as file:
        file.write(threads)

def save_mask(image, rotate=False):
    print("Trying to save mask as: " + get_save_id("mask"))
    canvas = (image / (image.max() / 255)).astype(np.uint8)
    if rotate:
        canvas = cv2.rotate(canvas, cv2.ROTATE_90_COUNTERCLOCKWISE)
    cv2.imwrite(get_save_id("mask"), canvas)

def save_original(image):
    print("Trying to save original image as: " + get_save_id("original"))
    cv2.imwrite(get_save_id("original"), image)

#
# Image
#
def decide_aspect_ratio(image):
    width, height = image.shape[1], image.shape[0]
    img_ar = width / height
    arr = np.asarray([1/1 , 3/4, 4/3, 4/5, 5/4])
    return arr[(np.abs(arr - img_ar)).argmin()]
    
def center_crop_to_aspect_ratio(image, aspect_ratio):
    width, height = image.shape[1], image.shape[0]
    img_ar = width / height
    if img_ar > aspect_ratio:
        return image[:, int((width - height * aspect_ratio) / 2):int((width + height * aspect_ratio) / 2)]
    else:
        return image[int((height - width / aspect_ratio) / 2):int((height + width / aspect_ratio) / 2), :]

def resize_to_aspect_ratio(image, aspect_ratio):
    if aspect_ratio == 1/1:
        return cv2.resize(image, (1561, 1561))
    elif aspect_ratio == 3/4:
        return cv2.resize(image, (1321, 1761))
    elif aspect_ratio == 4/3:
        return cv2.resize(image, (1761, 1321))
    elif aspect_ratio == 4/5:
        return cv2.resize(image, (1441, 1801))
    elif aspect_ratio == 5/4:
        return cv2.resize(image, (1801, 1441))

def get_input(): 
    input_image = cv2.imread(INPUT_IMAGE_PATH, 0)
    if INVERSE_INPUT_IMAGE:
        input_image = cv2.bitwise_not(input_image)

    aspect_ratio = decide_aspect_ratio(input_image)
    input_image = center_crop_to_aspect_ratio(input_image, aspect_ratio)
    input_image = resize_to_aspect_ratio(input_image, aspect_ratio)

    # Background removal
    is_success, buffor_array = cv2.imencode(".png", input_image)
    byte_image = buffor_array.tobytes()
    output_buffor_array = remove_background(byte_image)
    image_no_bg_alpha = cv2.imdecode(np.frombuffer(output_buffor_array, np.uint8), cv2.IMREAD_UNCHANGED)

    # Edge detection
    image_blur = cv2.GaussianBlur(input_image, (3, 3), 0)
    image_sobel_xy = cv2.convertScaleAbs(cv2.Sobel(src=image_blur, ddepth=cv2.CV_64F, dx=1, dy=1, ksize=5))
    image_edges = image_sobel_xy / (image_sobel_xy.max() / MASK_EDGES_FOCUS)

    # Mask creation
    mask = np.ones(image_edges.shape, np.float64) * (MASK_BACKGROUND_FOCUS + MASK_OBJECT_FOCUS + MASK_EDGES_FOCUS)
    for i in range(input_image.shape[0]):
        for j in range(input_image.shape[1]):
            if image_no_bg_alpha[i, j][3] <= 25:
                mask[i, j] = mask[i, j] - MASK_BACKGROUND_FOCUS
            else:
                mask[i, j] = mask[i, j] - (MASK_OBJECT_FOCUS + image_edges[i, j])

    # Contrast enhancement
    mean = np.uint8(cv2.mean(input_image)[0])
    input_image = cv2.addWeighted(input_image, CONTRAST_FACTOR, np.ones_like(input_image) * mean, 1-CONTRAST_FACTOR, 0.0)
    save_original(input_image)
    
    return input_image, mask, aspect_ratio

def get_new_image(dimensions):
    return np.ones(dimensions, np.uint8) * 255

def get_preffered_iterations(pixel_average):
    arrangement_param = 1.0
    if NAIL_ARRANGEMENT == "rectangle": arrangement_param = 4 / np.pi
    return int((500 + PREFERED_DARKNESS * 10 * (255-pixel_average)) * arrangement_param)

def get_iterations(pixel_average):
    if ITERATION_MODE == "interval":
        return get_preffered_iterations(pixel_average), MAX_ITERATIONS + 1
    elif ITERATION_MODE == "auto":
        pref_it = get_preffered_iterations(pixel_average)
        return pref_it, pref_it + SAVE_AROUND + 1
    return -1, -1

def decide_to_save(iteration_number, pref_it):
    if ITERATION_MODE == "interval":
        if iteration_number >= START_SAVE and iteration_number % SAVE_INTERVAL == 0:
            return True
    elif iteration_number == pref_it or \
         iteration_number == pref_it - SAVE_AROUND or \
         iteration_number == pref_it + SAVE_AROUND:
        return True
    return False
    
def draw_line(image, line, color):
    return cv2.line(image, (line[0][1], line[0][0]), (line[1][1], line[1][0]), color, 1)

def draw_point(image, point, color):
    image[point[1], point[0]] = color
    return image

#
# Points
#
def get_data_from_dictionaries(aspect_ratio, nail_arrangement):
    ar = ""
    rotate = False
    if aspect_ratio == 1/1:
        ar = "1x1"
    elif aspect_ratio == 3/4: 
        ar = "4x3"
    elif aspect_ratio == 4/3:
        ar = "4x3"
        rotate = True
    elif aspect_ratio == 4/5:
        ar = "5x4"
    elif aspect_ratio == 5/4:
        ar = "5x4"
        rotate = True

    with open("dictionaries/" + nail_arrangement + "_" + ar + ".dat", "rb") as dictionary_file:
        n, t, ntt, ttp = pickle.load(dictionary_file)

    return n, t, ntt, ttp, rotate


def get_other_end_of_thread(thread, nail):
    if np.array_equal(thread[0], nail):
        return thread[1]
    return thread[0]

def get_nail_number(nail, n):
    for i in range(len(n)):
        if np.array_equal(nail, n[i]):
            return i
    return -1


#
# Test 
#
# input_image = get_input_image()

# input_image_edges = get_input_image_edges()
# output_image = get_new_image(input_image.shape)



# print("Input image")
# display_image(input_image)

# print("Input image edges")
# display_image(input_image_edges)

# print("Output image")
# display_image(output_image)



In [ ]:
import string
import re
from tabulate import tabulate
from skimage.metrics import structural_similarity

def calculate_images_similarity_indexes(work_dir):
    # column_names = ["FOTO", "INFO", "MEAN", "MEAN DIFF", "MSE", "PSNR", "SSIM"]
    column_names = ["FILENAME", "KIND", "MEAN", "MEAN DIFF", "MSE", "PSNR", "SSIM"]
    values = []

    sift = cv2.SIFT_create()
    bf = cv2.BFMatcher()

    i = 0
    for filename in sorted(os.listdir(work_dir)):
        i += 1
        print(filename + " #" + str(i) + " of " + str(len(os.listdir(work_dir))))

        if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
            inilist_us = [m.start() for m in re.finditer(r'_', filename)]
            inilist_png = [m.start() for m in re.finditer(r'png', filename)]
            name = filename[:inilist_us[0]]
            kind = filename[inilist_us[0]+1:inilist_us[1]]

            original_image = cv2.imread(work_dir + filename, 0)
            image = cv2.imread(work_dir + filename, 0)

            if kind == "mask" or kind == "original":
                continue
            
            if kind == "e" or kind == "r" or \
                kind == "jenny" or kind == "callum" or \
                kind == "birsak" or kind == "xiaonan" or kind == "petros":
                # find file containing the same name
                for filename2 in sorted(os.listdir(work_dir)):
                    if name in filename2 and "original" in filename2:
                        original_image = cv2.imread(work_dir + filename2, 0)
                        break

            # info_str = ""
            # if kind == "e" or kind == "r":
            #     bg = filename[inilist_us[4]+1:inilist_us[5]]
            #     obj = filename[inilist_us[5]+1:inilist_us[6]]
            #     ed = filename[inilist_us[6]+1:inilist_png[0]-1]
            #     info_str = "\makecell{" + name + "\\\\" + "MBF = " + bg[2:] + "\\\\" + "OBJ = " + obj[3:] + "\\\\" + "ED = " + ed[2:] + "}"

            # resize image to width 200 preserving aspect ratio
            original_image = cv2.resize(original_image, (200, int(200 * original_image.shape[0] / original_image.shape[1])), interpolation=cv2.INTER_AREA)
            image = cv2.resize(image, (200, int(200 * image.shape[0] / image.shape[1])), interpolation=cv2.INTER_AREA)
            
            # foto_str = "\includegraphics[width=0.05\\textwidth]{img/6-comp/" + filename + "}"

            mean = round(np.mean(image), 2)
            mean_original = np.mean(original_image)
            mean_diff = round(mean - mean_original, 2)
            mean_diff_str = str(mean_diff)
            if mean_diff > 0:
                mean_diff_str = "+" + str(mean_diff)

            mse = round(np.mean((image - original_image) ** 2), 2)
            psnr = round(10 * np.log10(255 / mse), 2)
            ssim = round(structural_similarity(image, original_image), 2)
            # values.append([foto_str, info_str, mean, mean_diff_str, mse, psnr, ssim])
            values.append([filename, kind, mean, mean_diff_str, mse, psnr, ssim])


    tabulate.LATEX_ESCAPE_RULES = {}
    with open(work_dir + "/!similarity_indexes.txt", "w") as file:
        file.write(tabulate(values, headers=column_names))
        # file.write(tabulate(values, headers=column_names, tablefmt='latex'))

calculate_images_similarity_indexes("test_select/")

In [ ]:


def get_thread_panelty(thread, ttp, input_image, input_image_mask):
    image_t_vals = input_image[tuple(ttp[thread.tobytes()].T)] # values of pixels in thread in input image
    mask_t_vals = input_image_mask[tuple(ttp[thread.tobytes()].T)] # values of pixels in thread in mask image
    return np.average(image_t_vals, weights=mask_t_vals)

def get_best_fitting_thread(ntt, current_nail, ttp, input_image, input_image_edges): 
    current_threads = ntt[current_nail.tobytes()]
    best_fitting_thread = current_threads[0]
    best_fitting_thread_penalty = 999999
    for thread in current_threads:
        penalty = get_thread_panelty(thread, ttp, input_image, input_image_edges)
        if penalty < best_fitting_thread_penalty:
            best_fitting_thread = thread
            best_fitting_thread_penalty = penalty
    return best_fitting_thread, best_fitting_thread_penalty

def produce_thread_art():
    print("Producing " + NAIL_ARRANGEMENT + " thread art for: ", INPUT_IMAGE_PATH)
    
    input_image, input_image_mask, aspect_ratio = get_input()
    output_image = get_new_image(input_image.shape)
    thread_path = ""

    print("Parameters:  \tAspect Ratio: " + str(aspect_ratio) + 
                        "\t Mask BG: " + str(MASK_BACKGROUND_FOCUS) + 
                        "\t Mask OBJ: " + str(MASK_OBJECT_FOCUS) + 
                        "\t Mask EDGE: " + str(MASK_EDGES_FOCUS))

    n, t, ntt, ttp, rotate = get_data_from_dictionaries(aspect_ratio, NAIL_ARRANGEMENT)

    if rotate: # if dictionary exists for rotated image, use it
        input_image = cv2.rotate(input_image, cv2.ROTATE_90_CLOCKWISE)
        input_image_mask = cv2.rotate(input_image_mask, cv2.ROTATE_90_CLOCKWISE)
        output_image = cv2.rotate(output_image, cv2.ROTATE_90_CLOCKWISE)

    used_threads = []
    current_nail = ntt[list(ntt)[0]][0][0] # start at more suitable one?
    thread_path = thread_path + str(get_nail_number(current_nail, n)) + " "
    current_input_image = input_image.copy()

    save_mask(input_image_mask, rotate)
    if VISUALIZE_DURING_RUN:
        print("Input image: ")
        display_image(current_input_image, rotate)
        
        print("Input image mask: ")
        display_mask(input_image_mask, rotate)

    preffered_iterations, max_iterations = get_iterations(np.average(input_image_mask))
    for i in range(1, max_iterations):
        current_threads = ntt[current_nail.tobytes()]
        if len(current_threads) == 0:
            print("No threads for nail", current_nail)
            break 

        selected_thread, selected_thread_penalty = get_best_fitting_thread(ntt, current_nail, ttp, current_input_image, input_image_mask)
        next_nail = get_other_end_of_thread(selected_thread, current_nail)
        thread_path = thread_path + str(get_nail_number(next_nail, n)) + " "
        if i % 20 == 0:
            thread_path = thread_path + "\n"

        used_threads.append(selected_thread)

        ntt[current_nail.tobytes()] = np.array([a for a, skip in zip(ntt[current_nail.tobytes()], 
                                        [np.allclose(a, selected_thread) for a in ntt[current_nail.tobytes()]]) if not skip])
        ntt[next_nail.tobytes()] = np.array([a for a, skip in zip(ntt[next_nail.tobytes()], 
                                        [np.allclose(a, selected_thread) for a in ntt[next_nail.tobytes()]]) if not skip])
        
        if decide_to_save(i, preffered_iterations):
            print("Current nail #", i, ": ", str(current_nail))
            save_threads(used_threads, output_image, rotate)
            save_threads_svg(used_threads, output_image.shape, True)
            save_threads_path(thread_path)
            if VISUALIZE_DURING_RUN:
                display_threads(used_threads, output_image, rotate)
                display_image(current_input_image, rotate)

        draw_line(current_input_image, selected_thread, 255)
        current_nail = next_nail

    return used_threads

if CLEAN_WORKSPACE:
    clean_workspace()

# Run for all in DIR
# for filename in sorted(os.listdir("./img/")):
#     if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
#         INPUT_IMAGE_PATH = "img/" + filename
#         OUTPUT_IMAGE_PATH = "img_out/" + os.path.splitext(INPUT_IMAGE_PATH[4:])[0]   

#         CONTRAST_FACTOR = 1.0
#         a, b, c = get_input()
#         cv2.imwrite("org/" + filename, a)

# Run for one image specified in parameters
# threads_art = produce_thread_art()


#######
# TESTING:

# for filename in sorted(os.listdir("./img/")):
#     if filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".jpeg"):
#         INPUT_IMAGE_PATH = "img/" + filename
#         OUTPUT_IMAGE_PATH = "img_out/" + os.path.splitext(INPUT_IMAGE_PATH[4:])[0]
#         if filename != "simba.jpg":
#             continue
#         vals = np.array([
#             [1, 100, 5000], 
#             [1, 1000, 5000],
#             [1, 1000, 50000], 
#             [1, 1, 99999999]
#         ])

#         for i in range(vals.shape[0]):
#             MASK_BACKGROUND_FOCUS = vals[i][0]
#             MASK_OBJECT_FOCUS = vals[i][1]
#             MASK_EDGES_FOCUS = vals[i][2]
#             produce_thread_art()

